# You can save or load models using huggingface access token login

In [4]:
# from huggingface_hub import notebook_login

# notebook_login()

# Importing necessary libraries and dependecies


In [5]:
import json
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
import time
import numpy as np
from sklearn.metrics import accuracy_score

In [6]:
df = pd.read_csv("reader/train.csv")
df1 = pd.read_csv("reader/test.csv")

# For theme wise finetuning

In [12]:
# topics=[]
# df2 = df.groupby(['title'])['question'].count().reset_index(
#   name='Count').sort_values(['Count'], ascending=False)
# for i in df2['title'][:1]:
#     topics.append(i)

In [14]:
# topics

['New_York_City']

In [15]:
# df2

,title,Count
183,New_York_City,563
8,American_Idol,525
29,Beyoncé,523
98,Frédéric_Chopin,490
40,Buddhism,422
...,...,...
83,England_national_football_team,43
215,Race_and_ethnicity_in_the_United_States_Census,38
264,Tristan_da_Cunha,34
90,Federal_Aviation_Administration,30


# Preprocessing and bringing the dataset to usable format using generator functions

In [ ]:
df = df.loc[df['Answer_possible'] == True]
# df1 = df1.loc[df1['Answer_possible'] == True]
df = df.drop('Answer_possible', axis = 1)
df1 = df1.drop('Answer_possible', axis = 1)
df = df.reset_index()
df1 = df1.reset_index()
df = df.drop(columns="index", axis=1)
df1 = df1.drop(columns="index", axis=1)

In [16]:
from ast import literal_eval
listofdict = []
for i, j in enumerate(df['id']):
  x = dict()
  x['text'] = literal_eval(df['text'][i])
  x['answer_start'] = literal_eval(df['answer_start'][i])
  listofdict.append(x)
listofdict1 = []
for i, j in enumerate(df1['id']):
  x = dict()
  x['text'] = literal_eval(df1['text'][i])
  x['answer_start'] = literal_eval(df1['answer_start'][i])
  listofdict1.append(x)

In [17]:
df = df.drop('text', axis  = 1)
df1 = df1.drop('text', axis  = 1)
df = df.drop('answer_start', axis  = 1)
df1 = df1.drop('answer_start', axis  = 1)
df['answer'] = listofdict
df1['answer'] = listofdict1


In [22]:
def my_gen():
    for i,j in enumerate(df['id']):
        # if(df['title'][i] in topics):
        yield {"id": j, "title": df['title'][i], "context":df['context'][i], "question":df['question'][i], "answers": listofdict[i]}
def my_gen1():
    for i,j in enumerate(df1['id']):
        # if(df['title'][i] in topics):
        yield {"id": j, "title": df1['title'][i], "context":df1['context'][i], "question":df1['question'][i], "answers": listofdict1[i]}
dataset = Dataset.from_generator(my_gen)
test_dataset = Dataset.from_generator(my_gen1)

Using custom data configuration default-404d71d3096d32d5
Found cached dataset generator (/home/rishikesh_2001cs85/.cache/huggingface/datasets/generator/default-404d71d3096d32d5/0.0.0)
Using custom data configuration default-4e17f9908dddc3cc
Found cached dataset generator (/home/rishikesh_2001cs85/.cache/huggingface/datasets/generator/default-4e17f9908dddc3cc/0.0.0)


# Loading the tokenizer and creating mapping functions for train and validation datasets

In [25]:

tokenizer = AutoTokenizer.from_pretrained("deepset/minilm-uncased-squad2")

In [26]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if(len(answer['answer_start']) > 0):
          start_char = answer["answer_start"][0]
          end_char = answer["answer_start"][0] + len(answer["text"][0])
          sequence_ids = inputs.sequence_ids(i)
        else:
          start_char = 0
          end_char= 0
          sequence_ids = inputs.sequence_ids(i)
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not  completely inside the context passagge we label it as (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it is the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [27]:
dataset['answers'][0]['answer_start'][0]

526

In [29]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

In [30]:
tokenized_squad1 = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Loading cached processed dataset at /home/rishikesh_2001cs85/.cache/huggingface/datasets/generator/default-4e17f9908dddc3cc/0.0.0/cache-e1df7630841aef82.arrow


In [31]:
tokenized_squad2 = test_dataset.map(preprocess_validation_examples, batched=True, remove_columns=test_dataset.column_names)

  0%|          | 0/16 [00:00<?, ?ba/s]

# Loading the model and fine-tuning it 

In [32]:
data_collator = DefaultDataCollator()

In [34]:
model = AutoModelForQuestionAnswering.from_pretrained("deepset/minilm-uncased-squad2")

In [36]:
training_args = TrainingArguments(
    output_dir="minilm",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=400
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_squad,
    eval_dataset=tokenized_squad1,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.637300,3.213381
2,0.511100,3.188245
3,0.424400,3.072186


TrainOutput(global_step=2979, training_loss=0.5210520221547578, metrics={'train_runtime': 657.0966, 'train_samples_per_second': 181.256, 'train_steps_per_second': 4.534, 'total_flos': 5843660566851072.0, 'train_loss': 0.5210520221547578, 'epoch': 3.0})

In [ ]:
# model.push_to_hub(f"minilm-nb")
# tokenizer.push_to_hub(f"minilm-nb")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = tokenized_squad2.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to("cpu") for k in eval_set_for_model.column_names}
trained_model = model.to("cpu")

# Calculating results batchwise with size 1000

In [ ]:
batch1 = batch.copy()
batch1['input_ids'] = batch1['input_ids'][:1000]
batch1['token_type_ids'] = batch1['token_type_ids'][:1000]
batch1['attention_mask'] = batch1['attention_mask'][:1000]
batch1

In [ ]:
with torch.no_grad():
        outputs = trained_model(**batch1)

In [ ]:
import math
a = len(batch['input_ids'])/1000
b = int(len(batch['input_ids'])/1000)
x = math.ceil(1000*(a-b))

In [ ]:
for i, j in enumerate(range(int(len(batch['input_ids'])/1000))):
    if(i):
        i = i*1000
        batch1 = batch.copy()
        print(i)
        batch1['input_ids'] = batch1['input_ids'][i:(i+1000)]
        batch1['token_type_ids'] = batch1['token_type_ids'][i:(i+1000)]
        batch1['attention_mask'] = batch1['attention_mask'][i:(i+1000)]
        with torch.no_grad():
            outputs1 = trained_model(**batch1)
        outputs['start_logits'] = torch.cat((outputs['start_logits'],(outputs1['start_logits'])))
        outputs['end_logits'] = torch.cat((outputs['end_logits'],(outputs1['end_logits'])))

In [ ]:
batch1 = batch.copy()
print(len(batch1['input_ids']))
batch1['input_ids'] = batch1['input_ids'][b*1000:]
batch1['token_type_ids'] = batch1['token_type_ids'][b*1000:]
batch1['attention_mask'] = batch1['attention_mask'][b*1000:]
len(batch1['input_ids'])
with torch.no_grad():
    outputs1 = trained_model(**batch1)
outputs['start_logits'] = torch.cat((outputs['start_logits'],(outputs1['start_logits'])))
outputs['end_logits'] = torch.cat((outputs['end_logits'],(outputs1['end_logits'])))

# Calculating results for complete data at the same time

In [ ]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = "cpu"
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = model.to(device)
with torch.no_grad():
    outputs = trained_model(**batch)

# An efficient Tensor decoding strategy

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [ ]:
start_logits.shape

(453, 384)

In [ ]:
batch_size = start_logits.shape[0]
max_seq_len = start_logits.shape[1]

batch_size, max_seq_len

(3251, 384)

In [ ]:
start_matrix = start_logits.unsqueeze(2).expand(-1, -1, max_seq_len)
end_matrix = end_logits.unsqueeze(1).expand(-1, max_seq_len, -1)
# start_end_matrix = start_matrix + end_matrix
start_matrix.shape, end_matrix.shape

(torch.Size([3251, 384, 384]), torch.Size([3251, 384, 384]))

In [ ]:
start_end_matrix = start_matrix + end_matrix

In [ ]:
indices = torch.tril_indices(max_seq_len, max_seq_len, offset=-1, device=start_end_matrix.device)
start_end_matrix[:, indices[0][:], indices[1][:]] = -888

In [ ]:
max_answer_length = 1000

In [ ]:
indices_long_span = torch.triu_indices(
            max_seq_len, max_seq_len, offset=max_answer_length, device=start_end_matrix.device
        )
start_end_matrix[:, indices_long_span[0][:], indices_long_span[1][:]] = -777

In [ ]:
start_end_matrix[:, 0, 1:] = -666

In [ ]:
flat_scores = start_end_matrix.view(batch_size, -1)
flat_sorted_indices_2d = flat_scores.sort(descending=True)[1]
flat_sorted_indices = flat_sorted_indices_2d.unsqueeze(2)

In [ ]:
start_indices = torch.div(flat_sorted_indices, max_seq_len, rounding_mode="trunc")
end_indices = flat_sorted_indices % max_seq_len
sorted_candidates = torch.cat((start_indices, end_indices), dim=2)

In [ ]:
sorted_candidates.shape

torch.Size([3251, 147456, 2])

In [ ]:
duplicate_filtering = -1
n_best_per_sample = 5

In [ ]:
from predictions import QACandidate
from typing import List

In [ ]:
def get_top_candidates(sorted_candidates, start_end_matrix, sample_idx: int, start_matrix, end_matrix):
        """
        Returns top candidate answers as a list of Span objects. Operates on a matrix of summed start and end logits.
        This matrix corresponds to a single sample (includes special tokens, question tokens, passage tokens).
        This method always returns a list of len n_best + 1 (it is comprised of the n_best positive answers along with the one no_answer)
        """
        # Initialize some variables
        top_candidates: List[QACandidate] = []
        n_candidates = sorted_candidates.shape[0]
        start_idx_candidates = set()
        end_idx_candidates = set()

        start_matrix_softmax_start = torch.softmax(start_matrix[:, 0], dim=-1).cpu().numpy()
        end_matrix_softmax_end = torch.softmax(end_matrix[0, :], dim=-1).cpu().numpy()
        # Iterate over all candidates and break when we have all our n_best candidates
        for candidate_idx in range(n_candidates):
            # Retrieve candidate's indices
            start_idx = sorted_candidates[candidate_idx, 0]
            end_idx = sorted_candidates[candidate_idx, 1]
            # Ignore no_answer scores which will be extracted later in this method
            if start_idx == 0 and end_idx == 0:
                continue
            if duplicate_filtering > -1 and (start_idx in start_idx_candidates or end_idx in end_idx_candidates):
                continue
            score = start_end_matrix[start_idx, end_idx]
            confidence = (
                (start_matrix_softmax_start[start_idx] + end_matrix_softmax_end[end_idx]) / 2
                if score > -500
                else np.exp(score / 10)  # disqualify answers according to scores in logits_to_preds()
            )
            top_candidates.append(
                QACandidate(
                    offset_answer_start=start_idx,
                    offset_answer_end=end_idx,
                    score=score,
                    answer_type="span",
                    offset_unit="token",
                    aggregation_level="passage",
                    passage_id=str(sample_idx),
                    confidence=confidence,
                )
            )
            if duplicate_filtering > -1:
                for i in range(0, duplicate_filtering + 1):
                    start_idx_candidates.add(start_idx + i)
                    start_idx_candidates.add(start_idx - i)
                    end_idx_candidates.add(end_idx + i)
                    end_idx_candidates.add(end_idx - i)

            # Only check if we have enough candidates after adding new candidate to the list
            if len(top_candidates) == n_best_per_sample:
                break

        no_answer_score = start_end_matrix[0, 0]
        no_answer_confidence = (start_matrix_softmax_start[0] + end_matrix_softmax_end[0]) / 2
        top_candidates.append(
            QACandidate(
                offset_answer_start=0,
                offset_answer_end=0,
                score=no_answer_score,
                answer_type="no_answer",
                offset_unit="token",
                aggregation_level="passage",
                passage_id=None,
                confidence=no_answer_confidence,
            )
        )

        return top_candidates

In [ ]:
sorted_candidates = sorted_candidates.cpu().numpy()
start_end_matrix = start_end_matrix.cpu().numpy()

In [ ]:
all_top_n = []
for sample_idx in range(batch_size):
    sample_top_n = get_top_candidates(
        sorted_candidates[sample_idx],
        start_end_matrix[sample_idx],
        sample_idx,
        start_matrix=start_matrix[sample_idx],
        end_matrix=end_matrix[sample_idx],
    )
    all_top_n.append(sample_top_n)

In [ ]:
len(all_top_n)

3251

# O(n^2) decoding

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(tokenized_squad2):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
# start_logits = outputs.start_logits.cpu().numpy()
# end_logits = outputs.end_logits.cpu().numpy()

start_logits = outputs.start_logits.numpy()
end_logits = outputs.end_logits.numpy()

In [ ]:
offset_maps = tokenized_squad2["offset_mapping"]

In [ ]:
import numpy as np

n_best = 5
max_answer_length = 1000
predicted_answers = []

for example in test_dataset:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = offset_maps[feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    if (len(answers) == 0):
        predicted_answers.append({"id": example_id, "prediction_text": "", "conf_score": 0})
        continue
    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"], "conf_score": best_answer["logit_score"]})

100%|██████████| 450/450 [00:00<00:00, 8952.11it/s]


# O(nlogn) decoding

In [ ]:
import numpy as np
import bisect 
n_best = 20
max_answer_length = 30
predicted_answers = []

for example in test_dataset:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = offset_maps[feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        start_indexes1 = []
        end_indexes1 = []
        for i in start_indexes:
            if offsets[i] is None:
                continue
            else:
                start_indexes1.append(i)

        for i in end_indexes:
            if offsets[i] is None:
                continue
            else:
                end_indexes1.append(i)

        if((not len(start_indexes1)) or  (not len(end_indexes1))):
            answers.append(
                    {
                        "text": "",
                        "logit_score": 0.5,
                    }
            )
        start_indexes1.sort()
        end_indexes1.sort()
        ct = 0
        for start_index in start_indexes1:
            es = bisect.bisect_right(end_indexes1, start_index)
            ee = bisect.bisect_right(end_indexes1, start_index+ max_answer_length)
            for end_index_ind in range(es, ee):
                ct+=1
                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_indexes1[end_index_ind]][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_indexes1[end_index_ind]],
                    }
                )
        if(not ct):
            answers.append(
                    {
                        "text": "",
                        "logit_score": 0.5,
                    }
            )
    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"], "conf_score": best_answer["logit_score"]})

# Saving the results and calculating exact match for predictions(replace appropriate paths while using the code)

In [ ]:
dfx = pd.DataFrame()
dfx['id'] = [i['id'] for i in predicted_answers]
dfx['text'] = [i['prediction_text'] for i in predicted_answers]
dfx['conf_score'] = [i['conf_score'] for i in predicted_answers]
dfx.to_csv("minilm_pred_final_theme_pretrained_ind1.csv")

In [ ]:
df_pred = pd.read_csv("./minilm_pred_final_theme_pretrained_ind1.csv", index_col="id")
df_pred

,Unnamed: 0,text,conf_score
id,,,
10489,0,pantheistic religion,-3.565175
10490,1,"Buddhist, Hindu and Sikh",6.598208
10552,2,specific formal prescriptions or canons,12.181886
10458,3,very rapidly,12.992739
10545,4,religious or empirical,11.059700
...,...,...,...
14783,445,the use of recurring characters,5.111533
14901,446,Europe,-0.413015
14806,447,Punch,12.736288


In [ ]:
df_pred.fillna("", inplace=True)

In [ ]:
df_test = pd.read_csv("test.csv", index_col="id")
df_test = df_test.loc[df_pred.index.to_list()]

In [ ]:
ans_index = df_test.loc[df_test.Answer_possible == True].index
ans_index

Int64Index([10552, 10458, 10545, 10442, 10466, 11034, 11001, 10860, 10852,
            10944,
            ...
            14703, 14711, 14608, 14751, 14907, 15008, 14881, 14925, 14888,
            14806],
           dtype='int64', name='id', length=291)

In [ ]:
df_test["text"] = df_test["text"].apply(lambda x : x[2:-2])
df_test.head()

,title,context,question,Answer_possible,text,answer_start
id,,,,,,
10489,Architecture,The architecture of different parts of Asia de...,What religion led to architectural changes in...,False,,[]
10490,Architecture,The architecture of different parts of Asia de...,Which cultures architecture showed a lot of s...,False,,[]
10552,Architecture,Texts on architecture have been written since ...,Aside from general advice what did the texts c...,True,specific formal prescriptions,[105]
10458,Architecture,It is widely assumed that architectural succes...,At what speed did urban settlements sometimes ...,True,rapidly,[581]
10545,Architecture,The architecture and urbanism of the Classical...,What ideals were Greek and Roman Classical arc...,True,religious or empirical ones,[131]


In [ ]:
accuracy_score(df_pred.loc[ans_index].text, df_test.loc[ans_index].text) #pretrained

0.7814229249011858

In [ ]:
accuracy_score(df_pred.loc[ans_index].text, df_test.loc[ans_index].text) #finetuned

0.749802371541502

In [ ]:
accuracy_score(df_pred.loc[ans_index].text, df_test.loc[ans_index].text)

0.8178694158075601

In [ ]:
accuracy_score(df_pred.loc[ans_index].text, df_test.loc[ans_index].text)

0.8178694158075601